## Testing the deployment with sample data

### Library imports

In [2]:
from azure.identity import DefaultAzureCredential          
from azure.ai.ml import MLClient

import mlflow          
from mlflow.deployments import get_deploy_client 

import pandas as pd
import numpy as np
import json

### Get handle to workspace

In [5]:
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id="27a6aae6-ce60-4ae4-a06e-cfe9c1e824d4",
    resource_group_name="RG-ADA-MLOPS-POC",
    workspace_name="azu-ml-ada-mlops-poc",
)

endpoint_name = "MC-Prod-Endpoint-4242"

deployment_client = get_deploy_client(mlflow.get_tracking_uri())

### Get sample data

In [6]:
data_simulation = ml_client.data.get(name="data_simulation", version=2)

samples = (
    pd.read_csv(data_simulation.path, sep=";")
    .dropna() # UNTIL IMPUTATION IS IMPLEMENTED IN SCORING SCRIPT
    .sample(n=5)
    .drop(columns=["Response"])
    .reset_index(drop=True)
)

samples

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue
0,375,1954,Graduation,Divorced,76773.0,0,0,2013-02-25,79,516,...,11,1,0,0,0,0,0,0,3,11
1,922,1979,2n Cycle,Married,31086.0,1,1,2013-05-04,79,16,...,2,8,1,0,0,0,0,0,3,11
2,8147,1960,Graduation,Married,31454.0,1,1,2013-07-06,40,28,...,3,8,0,0,0,0,0,0,3,11
3,10675,1956,PhD,Married,66334.0,0,1,2013-04-03,82,909,...,5,5,0,0,0,0,0,0,3,11
4,2173,1952,Graduation,Widow,40049.0,0,1,2013-10-16,23,13,...,3,6,0,0,0,0,0,0,3,11


In [8]:
predictions = deployment_client.predict(endpoint=endpoint_name, df={"dataframe_split": samples.to_dict(orient='split')})
predictions

,0
0,0
1,0
2,0
3,1
4,0
